In [ ]:
s = 'GATATATGCATATACTT'
t = 'ATAT'

s = 'GTACTAATAATAAGACCTGTATCCCACTAATATCCCACTAATACGTCAACTAATAACTAATAACTAATACGGAACTAATAACTAATAGTGCTACTAATACAAAGGTCGTATACTAATAGCTACTAATAACTAATAACTAATACTTGACTAATAGGACTAATAGGACTAATAGTAGCCACTAGACTAATAAACTAATAACTAATACACTAATAGCAAGGTATACTAATACGTAACTAATATACGACTAATACACTAATAACTAATACGACTAATAAGAACACTAATACTAGTCCCGTCTACTAATAACTAATAAACTAATAACGGACTAATAAGTGCAACTAATAGCACGTAACTAATAACTAATATTTACAACTAATACAACTAATATGTAACACTAATACAGCCCACTAATACGACTAATATTCAGTACTAATAGATTAAAACTAATAATGCGTACTAATAGACTAATAAAACTAATATTCACTACTAATATAACTAATAGTAGGACTAATAGAACTAATAACGGTACTAATAGACTAATATCAGGGGTCCACTAATAACTAATATACTAATAACTAATAACTAATAACTAATAACTAATAGCGCCACTAATATGACTAATAGCGCTACTAATACACTAATAACTAATAACTAATAAACTAATAACTAATAAACTAATACTACTAATATTAACTAATAGACTAATAGCACTAATAGGCTACTAATAACTAATAATCACTAATACGCTACTAATAACTAATAGACTAATAATGGAAGTACTAATACTCTCCCCACTAATATGGTTTGGTACTAATAAAACTAATAACTAATAACTAATA'
t = 'ACTAATAAC'

k = len(t)
n_kmers = len(s) - k + 1

bases = ['A', 'T', 'G', 'C']

from itertools import product
all_kmers = [''.join(item) for item in list((product(bases, repeat=k)))]
frequency_dict = {}
for kmer in all_kmers:
  frequency_dict[kmer] = []

for i in range(n_kmers):
  kmer = s[i:i + k]
  frequency_dict[kmer].append(i + 1)

for item in frequency_dict[t]:
  print(item, end = ' ')

49 56 74 122 129 191 252 299 314 352 516 553 568 575 582 589 637 644 659 721 749 819 826 

In [35]:
import numpy as np
import numpy.random as rand
from scipy import stats

#np.random.seed(77)
translation_dict = {'A': 0, 'T': 1, 'G': 2, 'C': 3}

def score(dna_list):
  dna_list = [[char for char in dna] for dna in dna_list]
  modes = stats.mode(dna_list)[0][0]
  score = np.matrix([modes[i] != dna[i] for dna in dna_list for i in range(len(modes))]).sum()

  return score, ''.join(list(modes))

def profile(dna_list):
  dna_len = len(dna_list[0])
  t = len(dna_list)

  profile = np.ones((4, dna_len)) #mp.ones for pseudocounts
  for dna in dna_list:
    for i in range(dna_len):
      profile[translation_dict[dna[i]], i] += 1 
  
  profile = profile / (t + 4)
  return profile


def gibbs_sampler(dna, k ,t, n):
  dna_len = len(dna[0])
  n_kmers = dna_len - k + 1
  ind = rand.choice(range(n_kmers), t)

  motifs = [dna_list[i][ind[i]:ind[i]+k] for i in range(t)]
  best_motifs = motifs
  
  for i in range(n):
    num = rand.randint(t)
    #print(num)
    motif = motifs.pop(num)
    prof = profile(motifs)

    probabilities = []
    for j in range(n_kmers):
      kmer  = [translation_dict[ch] for ch in dna_list[num][j:j + k]]
      p = 1
      for h in range(k):
        p *= prof[kmer[h], h]

      probabilities.append(p)
    max = probabilities.index(np.max(probabilities))
    new_motif = dna_list[num][max:max + k]
    motifs.insert(num, new_motif)
    if score(motifs) < score(best_motifs):
      best_motifs = motifs

  return best_motifs

# dna_list = ['CGCCCCTCTCGGGGGTGTTCAGTAAACGGCCA', 
#       'GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG',
#       'TAGTACCGAGACCGAAAGAAGTATACAGGCGT',
#       'TAGATCAAGTTTCAGGTGCACGTCGGTGAACC',
#       'AATCCACCAGCTCCACGTGCAATGTTGGCCTA']
# k = 8
# t = 5
# n = 100

dna = open('/content/rosalind_ba2g.txt', 'r').readlines()

dna_list = [dna.replace('\n', '') for dna in dna[1:]]
k = int(dna[0][:2])
t = int(dna[0][3:5])
n = int(dna[0][6:])
print(f'k = {k}\nt = {t}\nn = {n}')

motifs = gibbs_sampler(dna_list, k, t ,n)
print(score(motifs))

for item in motifs:
  print(item)

k = 15
t = 20
n = 2000
(70, 'CAGTAGACGTTTTTA')
CGTCTGACGTTGTTA
GAGTTTTCGTTTTTA
CAACGGACGTTTTTA
GAGTATCGGTTTTTA
GAGTAGACGTTTGAT
CAGTAGCGCTTTTTA
CAGTAGATAGTTTTA
AGGTAGACGTTTTTG
CAGTAGACGTTCGAA
GAGTAGACGTACGTA
GAGAGTACGTTTTTA
CAGTAGACGCGCTTA
AAGTAATGGTTTTTA
CAGTAGCGCTTTTTA
TTTCAGACGTTTTTA
AAGTAGACACCTTTA
CAGTAGACGTTTTCC
AAGTTTTCGTTTTTA
CAGTAGAAAGTTTTA
GCTTAGACGTTTTTA
